In [188]:
from pymongo import MongoClient
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
import statsmodels.api as sm
from collections import Counter

In [187]:
twitter = Twitter()

In [3]:
english = re.compile(r'[a-zA-Z0-9]')

In [4]:
korean = re.compile(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')

In [178]:
mongo = MongoClient(mongo_server, 27017)
review = mongo.restaurant.review

# 서버에서 DB 불러오기

In [7]:
good_temp_list = []

In [8]:
for record in review.find({'$and' : [{'rating' : '맛있다'},{'len_review' : {'$gte': 50}}]}):
    text = re.sub(english,'',record['review'])
    if re.search(korean, text.encode('utf-8')):
        good_temp_list.append(text)

In [9]:
print len(good_temp_list)

11919


In [70]:
bad_temp_list = []

In [71]:
for record in review.find({'$and' : [{'rating' : '별로'},{'len_review' : {'$gte': 50}}]}):
    text = re.sub(english,'',record['review'])
    if re.search(korean, text.encode('utf-8')):
        bad_temp_list.append(text)

In [72]:
print len(bad_temp_list)

1311


In [179]:
pipelines = []
pipelines.append({'$group': {'_id': '$restaurant_name','review':{'$addToSet':'$review'}}})
result = review.aggregate(pipelines)

In [180]:
seoul_restaurant_reviews = []
for doc in result:
    seoul_restaurant_reviews.append(doc)

# 전처리 - stemming

In [85]:
def do_stemming(reviews):
    result_list = []
    for i in range(len(reviews)):
        a = re.sub(' ','s',reviews[i])    #띄어쓰기 위해 s 붙힘
        b = twitter.pos(a, stem = True, norm= True)  #형태소 분석
        string = ''
        for word in b:                #문장으로 합침
            string += word[0]
        c = re.sub('s',' ',string)    #띄어쓰기 분리
        result_list.append(c)
    return result_list

In [90]:
result_list = do_stemming(good_temp_list)

In [91]:
bad_result_list = do_stemming(bad_temp_list)

In [92]:
len(result_list)

11919

In [93]:
len(bad_result_list)

1311

In [100]:
service_good_feature = {'서비스':['좋','친절','괜찮','최고','빠르','짱','훌륭','추천','감사','구수','최상','대박','훈훈','특별','개이득','최고','만족','세련','최고','감동'],'사장':['친절','스윗','센스'],'알바':['친절','스윗','센스'],'직원': ['친절','스윗','센스']}

In [192]:
service_bad_feature = {'서비스': ['아쉽','최악','나쁘','느리','빡치','비추','별로','그냥','낙제','쏘다쏘다','엉망','실망','불친절','문제','컴플레인','거지','그닥','그다지','구려','불편','엉성','헬','개판'],'알바':['불친절','똑바로','재수'],'사장':['불친절','똑바로','재수'],'직원':['불친절','똑바로','재수']}

In [102]:
taste_good_feature = {'간이':['맞','적절','딱','환상','담백'],'간도':['맞','적절','딱','환상','담백'],'음식':['깔끔'],'맛':['있']}

In [191]:
taste_bad_feature = {'간이':['세','쎄','강하다','별로'],'간도':['세','쎄','강하다','별로'],'음식':['별로','쏘다쏘다','최악'],'맛':['별로','최악']}

In [104]:
taste_good_emotion = ['고소','부드럽','신선','촉촉','싱싱','정갈']

In [105]:
taste_bad_emotion = ['싱겁','느끼다하다','짜다','느끼다','짜다','딱딱하다','차갑다']

In [106]:
cost_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다','무난','괜춘','최상','최상','굿','엄지','낮'],'가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다','무난','괜춘','최상','최상','굿','엄지'],'양도':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든'],'양이':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든'],'양은':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든']}

In [190]:
cost_bad_feature ={'가격': ['비싸','있다','나쁘','사악','비효율','높다','부담','아쉽','쏘다쏘다','별로','그닥','그다지','쎄','ㅎㄷㄷ','높','거품'],'가성비':['별로'],'양이':['적다','작다','아쉽','적다','다소','별로'],'양은':['적다','작다','아쉽','적다','다소','별로'],'양도':['적다','작다','아쉽','적다','다소','별로']}

In [109]:
atmosphere_good_feature = {'분위기': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱','심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'], '인테리어': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱','심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근']}

In [110]:
atmosphere_bad_feature = {'분위기': ['나쁘다','바쁘다','어수선하다','이상하다','촌스럽다','별로','부담스럽다','시끄럽','복잡' ],'인테리어':[]}

In [111]:
visit_good_feature = {'의사':['있','충만','백프로','백프롭','많','만땅','마구','그득','만점','넘침'],'다시':['가다','오다','방문','찾다','가보다','한번','갈다','찾아가다','가야지','갈거다','방문하다보고','생각나다','방문한다면','와보고','재방문','접하다','간다면','갈다때가','먹다고프다','방문한다임','오자고','가기로','갈다생각이다','가면'],'굳이':[]}

In [112]:
visit_bad_feature = {'의사':['글쎄'],'굳이':['다시','많이','여기까지','줄서서','찾아','시키다','가다','찾다','여기','기다리다','줄을','사먹'],'다시':[]}

In [114]:
negative_word_emotion = ['안','않','못','없','아닌','아니']

In [134]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review.encode('utf-8')):
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))

    return feature_temp

In [131]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [132]:
def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

# Test 1: 리뷰별로 감정 분석

In [ ]:
for i, review in enumerate(bad_result_list):
    service_temp = get_feature_keywords(service_good_feature.keys(), review)
    good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

    atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
    good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)
    
    cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
    good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)
    
    visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
    good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)
    
    taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
    good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
    taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
    taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
    good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
    good_taste.extend(good_taste2)
    bad_taste.extend(bad_taste2)
    
    print i
    print 'good service emotions :{}'.format(len(good_service))
    for ngram in good_service:
        print ngram
    print 'bad service emotions :{}'.format(len(bad_service))
    for ngram in bad_service:
        print ngram
    print 'good atmosphere emotions :{}'.format(len(good_atmosphere))
    for ngram in good_atmosphere:
        print ngram
    print 'bad atmosphere emotions :{}'.format(len(bad_atmosphere))
    for ngram in bad_atmosphere:
        print ngram
    print 'good cost emotions :{}'.format(len(good_cost))
    for ngram in good_cost:
        print ngram
    print 'bad cost emotions :{}'.format(len(bad_cost))
    for ngram in bad_cost:
        print ngram
    print 'good visit emotions :{}'.format(len(good_visit))
    for ngram in good_visit:
        print ngram
    print 'bad visit emotions :{}'.format(len(bad_visit))
    for ngram in bad_visit:
        print ngram
    print 'good taste emotions :{}'.format(len(good_taste))
    for ngram in good_taste:
        print ngram
    print 'bad taste emotions :{}'.format(len(bad_taste))
    for ngram in bad_taste:
        print ngram
    print 

# Test 2. 서울내 음식점별로 감정 분석

In [196]:
for reviews in seoul_restaurant_reviews[:2]:
    print reviews['_id']
    reviews_list = do_stemming(reviews['review'])
    for i, review in enumerate(reviews_list):
        print review
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2)
        bad_taste.extend(bad_taste2)

        print i
        print 'good service emotions :{}'.format(len(good_service))
        for ngram in good_service:
            print ngram
        print 'bad service emotions :{}'.format(len(bad_service))
        for ngram in bad_service:
            print ngram
        print 'good atmosphere emotions :{}'.format(len(good_atmosphere))
        for ngram in good_atmosphere:
            print ngram
        print 'bad atmosphere emotions :{}'.format(len(bad_atmosphere))
        for ngram in bad_atmosphere:
            print ngram
        print 'good cost emotions :{}'.format(len(good_cost))
        for ngram in good_cost:
            print ngram
        print 'bad cost emotions :{}'.format(len(bad_cost))
        for ngram in bad_cost:
            print ngram
        print 'good visit emotions :{}'.format(len(good_visit))
        for ngram in good_visit:
            print ngram
        print 'bad visit emotions :{}'.format(len(bad_visit))
        for ngram in bad_visit:
            print ngram
        print 'good taste emotions :{}'.format(len(good_taste))
        for ngram in good_taste:
            print ngram
        print 'bad taste emotions :{}'.format(len(bad_taste))
        for ngram in bad_taste:
            print ngram
        print 

베테랑분식-센트럴시티 터미널
칼국수.. 굿 쫄면... 그냥 평범 만두... 굿
0
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :0

여기 맛남. 예전엔 별로이다 마느님이 맛있다 하다 나다.....
1
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :2
맛있다 하다 나다
마느님이 맛있다
bad taste emotions :0

칼국수가 얼마나 마싯다 하다 완전마싯다...ㅠㅠ너무마싯다ㅠㅠ음식도 빨리나오다 좋다ㅎㅎ쫄면이나 만두 등 나머지 메뉴는 그냥 저냥 평범하다 맛!
2
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :0

터미널에서 기다리다 우연히 먹다 이렇게 유명하다 곳인지 몰르다~ 모르다 먹다 맛있다! 자주 가다 최근에는 안 먹다 만두를 시키다보다 가성비다 별로고 맛은 정말 맛있다!
3
good s

만두를 좋아하다 만두도 시키다 되게 당면이 많다. 당면때문에 목이 메일지경ㅋㅋ당면이 모든 수분을 흡수하다. 만두는 촉촉하다 푸슬푸슬하다. 칼국수는 면이 부드럽다 후루룩 후루룩 자다 넘어가다. 일반 칼국수면에 비하다 얇다 원형(?)의 모양이구요. 국물은 걸쭉하다 계란 맛이 강하다~ 먹다보다 계란 푼 진라면순하다맛 국물(?)맛이 연상되다 느낌이랄끄다. 확실하다 겨울에 문득 생각나다 맛이라는 거! ㅋㅋ
33
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :3
부드럽다 후루룩 후루룩
면이 부드럽다
만두는 촉촉하다
bad taste emotions :0

숨기다 분식. 쫄면! 식초맛 살짝 나는 고추장쏘다스! 칼국수 먹기 전에 쫄면으로 입맛 달다
34
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :0

데블스도어-파미에가든
맛있다 엄청비싸다
0
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions 

26
good service emotions :0
bad service emotions :0
good atmosphere emotions :1
방문하다곳인데 분위기도좋다
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :3
맛이있다 맛있다 먹다오다
특유 맛이있다
부드럽다 특유 맛이있다
bad taste emotions :0

이태원을 중심으로 발생하다 크래프트비어 열풍이 신세계 정용진 회장 마음에 들다..양조장 하나를 차리다 음식점을 만들다 생각을 하다 신세계가 이렇다 면에서는 발 빠르다 트렌드를 캐치업 하다 것 같다. 매장 입구에선 별의 별 맥주들을 다 팔다 매장으로 가지고 들어가다 수는 없다 하다. 피자는 토마토소스가 풍부하다 좋다 프라이드맥앤치즈는 맛없다기가 오히려 어러운 음식이고 버거도 웬만하다 수제버거집들보다 괜찮다. 맥주는 에일도 좋아하다 반주로는 라거가 훨씬 적합하다 생각하다 사람인지라.. 라거도 좀 있다 좋다.
27
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :2
맛없다기가 오히려 어러운
프라이드맥앤치즈는 맛없다기가

분위기가 영국인줄..! 일단 분위기가 크다 맥주 맛이나 기본적인 음식 맛은 나쁘다 않다. 프라이와 샐러드 시키다 가격대는 꽤 있다 편이네요~ 프라이 7천원대 멕시칸 치킨 만원대~ 양이 적다 샐러드 많이 먹다 싶다 분은 카페마마스로 ㅎㅎ
28
good service emoti

good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :0

1. 페일 에일(아이리쉬 잔) : 시면서 쓰다쓸다 맛이에요. 2. 헬리스 : 친구가 먹다^^; 생략~ 아이리쉬 다음 크다 사이즈로 시키다 거인데 잔이 이쁘다. 3. 치킨윙과 새다리 : 음식 엄청 짜다 향신료 강하다. 중독성 있다 계속 물과 맥주를 찾다 되다 맛이에요. 양은 적다. / 친구 말에 의하다 여기다 음식이 전체적으로 짜다 맛이 강하다 하다~ 맥주2잔 안주1개가 총 26,000원 나오다 가볍다 2차 장소로 좋다 * 후기들 보다 웨이팅 길다 하다 저는 평일(월) 저녁 8시 넘어서다 가다 바로 들어가다(다 차고 2테이블 남다 상태이다). 사람이 적다 평일에 이용해보다 것도 방법일 것 같다.
47
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
bad taste emotions :4
짜다 향신료 강하다
짜다 맛이 강하다
짜다 향신료 강하다
짜다 맛이 강하다

점심 때 방문하다 이젠 대기가 전혀 없다 오히려 조명도 침침하니 약간 썰렁하다 느낌. 음식이 고루 맛나다 고구마후라이 강추.
48
good service emotions :0
bad service emotions :0
good atmosphere emotions :0
bad atmosphere emotions :0
good cost emotions :0
bad cost emotions :0
good visit emotions :0
bad visit emotions :0
good taste emotions :0
ba

# Test 3. 음식점 별로 속성별 감정지수 파악

In [195]:
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)

for i, reviews in enumerate(seoul_restaurant_reviews[:30]):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print i, reviews['_id']
    reviews_list = do_stemming(reviews['review'])
    for j, review in enumerate(reviews_list):
        #print review
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2)
        bad_taste.extend(bad_taste2)

        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
    
    print 'Total review count: {}'.format(len(reviews_list))
    print 'Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count) )
    print 'Bad service: {}/{} = {}'.format(restaurant_bad_service_count,restaurant_good_service_count + restaurant_bad_service_count, check_division(restaurant_bad_service_count, restaurant_good_service_count + restaurant_bad_service_count))
    print 'Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count)) 
    print 'Bad atmosphere: {}/{} = {}'.format(restaurant_bad_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count, check_division(restaurant_bad_atmosphere_count ,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))
    print 'Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count))
    print 'Bad cost: {}/{} = {}'.format(restaurant_bad_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, check_division(restaurant_bad_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count))
    print 'Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count))
    print 'Bad taste: {}/{} = {}'.format(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count))
    print

0 베테랑분식-센트럴시티 터미널
Total review count: 35
Good service: 1/2 = 0.5
Bad service: 1/2 = 0.5
Good atmosphere: 0/0 = 0
Bad atmosphere: 0/0 = 0
Good cost: 1/5 = 0.2
Bad cost: 4/5 = 0.8
Good taste: 18/21 = 0.86
Bad taste: 3/21 = 0.14

1 데블스도어-파미에가든
Total review count: 67
Good service: 1/1 = 1.0
Bad service: 0/1 = 0.0
Good atmosphere: 12/15 = 0.8
Bad atmosphere: 3/15 = 0.2
Good cost: 3/14 = 0.21
Bad cost: 11/14 = 0.79
Good taste: 24/32 = 0.75
Bad taste: 8/32 = 0.25

2 톰볼라-서래점
Total review count: 25
Good service: 2/3 = 0.67
Bad service: 1/3 = 0.33
Good atmosphere: 3/4 = 0.75
Bad atmosphere: 1/4 = 0.25
Good cost: 2/4 = 0.5
Bad cost: 2/4 = 0.5
Good taste: 7/9 = 0.78
Bad taste: 2/9 = 0.22

3 브루클린더버거조인트-본점
Total review count: 83
Good service: 0/1 = 0.0
Bad service: 1/1 = 1.0
Good atmosphere: 1/1 = 1.0
Bad atmosphere: 0/1 = 0.0
Good cost: 4/4 = 1.0
Bad cost: 0/4 = 0.0
Good taste: 28/33 = 0.85
Bad taste: 5/33 = 0.15

4 구슬함박-홍대본점
Total review count: 32
Good service: 0/0 = 0
Bad service: 0/0 = 0
Good at